In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

In [ ]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

#get test data
train_x = Variable(torch.unsqueeze(train_dataset.data,dim=1)).type(torch.FloatTensor)[2000:]/255.
train_y = train_dataset.targets[2000:]
test_x = Variable(torch.unsqueeze(test_dataset.data,dim=1)).type(torch.FloatTensor)[:2000]/255.
test_y = test_dataset.targets[:2000]


In [ ]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(    
                in_channels=1,
                out_channels=16,
                kernel_size=5,
                stride=1,
                padding=2,
            ),   
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16,32,5,1,2),
            nn.ReLU(),
            nn.MaxPool2d(2)

        )
        self.out = nn.Linear(32*7*7,10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)   
        x = x.view(x.size(0),-1) 
        output = self.out(x)
        return output


    
model = SimpleNet()
# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

In [ ]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for images,labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        b_x = Variable(images)
        b_y = Variable(labels)

        out = model(b_x)
        loss = criterion(out,b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset    
    test_out = model(test_x)
    train_out = model(train_x)
    pred_train_y = torch.max(train_out,1)[1].data.squeeze()
    pred_test_y = torch.max(test_out,1)[1].data.squeeze()
    
    train_accuracy = float(sum(pred_train_y == train_y)) / float(sum(train_y == train_y))    
    test_accuracy = float(sum(pred_test_y == test_y))/ float(sum(test_y == test_y))
    
    print('Epoch:',epoch,'---train accuracy:'+str(train_accuracy),'   ---test accuracy:'+str(test_accuracy))

    